### This code requires an input of a google API key and a channel ID for a YouTube channel. It will return a dataframe of all the videos on that channel along with that video's stats.

In [1]:
import pandas as pd
from googleapiclient.discovery import build
from datetime import datetime
from pandas.io.json import json_normalize
import json


api_key = Enter API Key Here
youtube = build('youtube','v3', developerKey = api_key)
start_time = datetime(year=2017, month=6, day = 1).strftime('%Y-%m-%dT%H:%M:%SZ')
end_time = datetime(year=2018, month=1, day = 1).strftime('%Y-%m-%dT%H:%M:%SZ')
channelID = 'UCu8ebUHoKbhVY0tavNdhwvw'
pd.set_option('max_columns', 300)
pd.set_option('max_rows', 5000)


def full_detail(channel_id):

    def get_channel_videos(channel_id):
        res = youtube.channels().list(id=channel_id, 
                                  part='contentDetails').execute()
        playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']

        videos = []
        next_page_token = None

        while 1:
            res = youtube.playlistItems().list(playlistId=playlist_id,
                                part='snippet',
                                maxResults=50,
                                pageToken=next_page_token).execute()

            videos +=  res['items']
            next_page_token = res.get('nextPageToken')

            if next_page_token is None:
                break

        return videos

    videos = get_channel_videos(channel_id)
    video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))

    def get_video_details(video_ids):
        details = []
        for i in range(0, len(video_ids), 50):
            res = youtube.videos().list(id=','.join(video_ids[i:i+50]),
                                      part='snippet,contentDetails,statistics',
                                       ).execute()
            details += res['items']

        return details

    fullDetails = get_video_details(video_ids)
    fullDetails = pd.json_normalize(fullDetails)

    fullDetails = fullDetails[['id','snippet.channelId','contentDetails.duration','snippet.publishedAt','snippet.title','snippet.description','statistics.commentCount','statistics.dislikeCount','statistics.likeCount','statistics.viewCount']]

    for each in range(len(fullDetails['contentDetails.duration'])):
        fullDetails['contentDetails.duration'][each] = fullDetails['contentDetails.duration'][each].replace('PT','').replace('M',':').replace('S','')

    fullDetails['date'] = fullDetails['id']
    for each in range(len(fullDetails['date'])):
        fullDetails['date'][each] = fullDetails['snippet.publishedAt'][each][:10]

    fullDetails['time'] = fullDetails['id']
    for each in range(len(fullDetails)):
        fullDetails['time'][each] = fullDetails['snippet.publishedAt'][each][-13:-6]

    better_cols = {}
    for each in fullDetails.columns:
        x = each.split('.')
        better_cols[each] = x[-1]
    fullDetails = fullDetails.rename(columns = better_cols)
    fullDetails = fullDetails[['id', 'channelId', 'duration','date','time' ,'publishedAt', 'title', 'description',
           'commentCount', 'dislikeCount', 'likeCount', 'viewCount']]
    return fullDetails
#fullDetails.to_excel('Full_Details.xlsx') #If you want to export to Excel, remove #

In [2]:
abl = full_detail('UCu8ebUHoKbhVY0tavNdhwvw')
abl.head()

,id,channelId,duration,date,time,publishedAt,title,description,commentCount,dislikeCount,likeCount,viewCount
0,W_SIL2cFJCM,UCu8ebUHoKbhVY0tavNdhwvw,14:28,2021-09-23,-23T21:,2021-09-23T21:30:04Z,UPDATE: Did Haitians Get WHIPPED By The Border...,The United States Border Patrol is facing alle...,484,19,1811,6919
1,PkrJK1U2_Fc,UCu8ebUHoKbhVY0tavNdhwvw,3H6:45,2021-09-23,-23T03:,2021-09-23T03:27:02Z,"🔴 ABL LIVE: Seth Rogen, HHS Whistleblower, Joy...",Call in! 434-658-1220. On tonight's episode of...,143,23,2602,22977
2,IYsSiY-L3EE,UCu8ebUHoKbhVY0tavNdhwvw,12:19,2021-09-22,-22T21:,2021-09-22T21:30:00Z,Bill Gates Gets NERVOUS During This Interview ...,Microsoft co-founder and philanthropist Bill G...,1641,24,7656,46746
3,EG_2xr-V6Xo,UCu8ebUHoKbhVY0tavNdhwvw,12:49,2021-09-22,-22T16:,2021-09-22T16:36:38Z,"Joy Reid Calls The Gabby Petito Case ""Missing ...","Joy Reid of MSNBC ""The Reidout"" referred to me...",1859,47,4725,32977
4,kDw_ozuRBc0,UCu8ebUHoKbhVY0tavNdhwvw,13:33,2021-09-21,-21T21:,2021-09-21T21:30:03Z,Black Lives Matter NYC Protests Restaurant Ove...,Black Lives Matter NYC staged a protest agains...,2765,220,9140,69550
